In [ ]:
import json
import numpy as np
from scipy.signal import butter, filtfilt, medfilt
import os

base_input_dir = ## Set input directory ##
base_output_dir = ## Set output directory ##

os.makedirs(base_output_dir, exist_ok=True)

def remove_outliers(data, kernel_size=3):
    return medfilt(data, kernel_size=kernel_size)

def butter_bandpass_filter(data, lowcut=0.5, highcut=8, fs=100, order=3):
    if len(data) < 2:
        return data
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, data)

def normalize_to_positive_range(data, target_min=30, target_max=55):
    data_shifted = data - np.min(data) + 1e-6
    data_min = np.min(data_shifted)
    data_max = np.max(data_shifted)
    if data_max == data_min:
        return np.full_like(data, target_min)
    normalized = target_min + (data_shifted - data_min) * (target_max - target_min) / (data_max - data_min)
    return normalized

json_files = [f for f in os.listdir(base_input_dir) if f.endswith('.json')]

for json_file in json_files:
    input_path = os.path.join(base_input_dir, json_file)
    output_path = os.path.join(base_output_dir, f"Cleaned_{json_file}")
    
    try:
        with open(input_path, "r") as f:
            raw_data = json.load(f)
        
        cleaned_data = {}
        
        for case_id, values in raw_data.items():
            try:
                arr = np.array(values)
                if len(arr) == 0:
                    continue
                
                arr = remove_outliers(arr)
                arr = butter_bandpass_filter(arr)
                arr = normalize_to_positive_range(arr)
                
                cleaned_data[case_id] = arr.tolist()
                
            except Exception as e:
                print(f"Error in case {case_id} ({json_file}): {e}")
                continue
        
        with open(output_path, "w") as f:
            json.dump(cleaned_data, f, indent=2)
        
        print(f"Processed {json_file} -> Cleaned_{json_file}")
        
    except Exception as e:
        print(f"Failed to read {json_file}: {e}")

print("Done")